In [1]:
import sqlite3
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import datetime as dt
from urllib.parse import urlparse

In [2]:
#accessing the browsing hsitory items

#locate the browsing history database
history_db_path = r'C:\Users\xjing\AppData\Local\Google\Chrome\User Data\Profile 2\History'

# Create a temporary copy of the database to avoid issues with running Chrome at the same time
temp_history_db_path = r'C:\Users\xjing\AppData\Local\Temp\History_Copy'
shutil.copyfile(history_db_path, temp_history_db_path)

#connect to the SQLite database
conn = sqlite3.connect(temp_history_db_path)
cursor = conn.cursor()

#sql query for obtaining the url,url title and visit time for all browsing history
query = """
    SELECT visits.id, urls.url, urls.title, visits.visit_time
    FROM visits
    INNER JOIN urls ON visits.url = urls.id
    LIMIT 1500
    """
#read the obtained data into a dataframe
df = pd.read_sql_query(query, conn)

# convert the visit_time to readable format
def convert_time(timestamp):
    dt = datetime.datetime(1601, 1, 1) + datetime.timedelta(microseconds=timestamp)
    return dt.replace(microsecond=0)

# apply the function to the 'visit_time' column
df['visit_time'] = df['visit_time'].apply(convert_time)

# Close the database connection
conn.close()

# Remove temporary copy
os.remove(temp_history_db_path)

In [3]:
#filter by day - list view

df['visit_date'] = df['visit_time'].dt.date
df.sort_values(['visit_date', 'visit_time'], inplace=True)

# group by 'visit_date'
grouped_by_day = df.groupby('visit_date')

# Loop through each group and print out the records for that day
for name, group in grouped_by_day:
    print(f"Date:{name}")
    print("="*50)  #add separator between days
    for index, row in group.iterrows():
        print(f"{row['url']},{row['title']}, {row['visit_time'].time()}")
    print()

Date:2023-08-14
https://www.google.com/search?q=chrome+extension&oq=chrome+extension+&aqs=chrome..69i57j0i512j0i433i512j0i512l7.3788j0j7&sourceid=chrome&ie=UTF-8,chrome extension - Google Search, 09:49:56
https://www.google.com/search?q=chrome+extension&oq=chrome+extension+&aqs=chrome..69i57j0i512j0i433i512j0i512l7.3788j0j7&sourceid=chrome&ie=UTF-8,chrome extension - Google Search, 09:49:56
https://www.google.com/search?q=how+to+develop+a+chrome+extension&sca_esv=556704752&ei=w_jZZLC4DdSQseMPx7CcqA0&ved=0ahUKEwjwpcPg79uAAxVUSGwGHUcYB9UQ4dUDCA8&uact=5&oq=how+to+develop+a+chrome+extension&gs_lp=Egxnd3Mtd2l6LXNlcnAiIWhvdyB0byBkZXZlbG9wIGEgY2hyb21lIGV4dGVuc2lvbjIIEAAYigUYkQIyBhAAGAcYHjIGEAAYCBgeMggQABiKBRiGAzIIEAAYigUYhgMyCBAAGIoFGIYDMggQABiKBRiGA0jcGVC8A1iYGHABeAGQAQCYAdoBoAHHFqoBBjAuMTQuM7gBA8gBAPgBAcICChAAGEcY1gQYsAPCAgoQABiKBRiwAxhDwgIIEAAYCBgHGB7iAwQYACBBiAYBkAYK&sclient=gws-wiz-serp,how to develop a chrome extension - Google Search, 09:50:01
https://www.google.com/search?q=how+to+dev

In [5]:
#filter by day - pie chart view
#input -- date
#output -- a pie chart showing proportions of visited domains on that day

# Let user enter a date
user_date_input = input('Enter a date in YYYY-MM-DD format: ')
try:
    user_date = dt.strptime(user_date_input, '%Y-%m-%d').date()
except ValueError:
    print("Invalid date format. Please enter a valid date in YYYY-MM-DD format.")
    exit()

# Filter the DataFrame based on this date
filtered_df = df[df['visit_date'] == user_date]

# Debugging Step: Check if the DataFrame is empty after filtering
if filtered_df.empty:
    print(f"No browsing history found for {user_date}.")
    exit()

# Parse the domain from URLs
filtered_df = filtered_df.copy()
filtered_df['domain'] = filtered_df['url'].apply(lambda x: urlparse(x).netloc)


# Count the number of visits for each domain
domain_count = filtered_df['domain'].value_counts()

# Debugging Step: Check if domain_count is empty
if domain_count.empty:
    print(f"No domain data available for {user_date}.")
    exit()

# Generate the pie chart
plt.figure(figsize=(10, 10))
plt.pie(domain_count, labels=domain_count.index, autopct='%1.1f%%')
plt.title(f"Number of visits per domain on {user_date}")
plt.show()


KeyboardInterrupt: Interrupted by user